In [113]:
import mysql.connector
import time
import datetime
import pandas as pd

In [114]:
df = pd.read_csv(r'solar-predictions.csv')
df = df.drop('Unnamed: 0', axis=1)
df

,forecastPeriod,temperature,feelsLike,weatherType,windSpeed,uvindex,precipitationProbability,windDirection,visibility,AO_PV_generation,Day sin,Day cos,Year sin,Year cos,Predictions
0,2019-09-01 00:00:00,14,14,7,2,0,4,14,4,0.00,-8.223106e-12,1.000000e+00,-0.862760,-0.505613,10.819783
1,2019-09-01 03:00:00,13,13,7,2,0,4,7,4,0.04,7.071068e-01,7.071068e-01,-0.863845,-0.503757,34.553062
2,2019-09-01 06:00:00,12,12,1,2,1,0,14,1,13.06,1.000000e+00,1.078645e-11,-0.864927,-0.501898,32.992691
3,2019-09-01 09:00:00,15,13,3,9,3,1,14,4,32.11,7.071068e-01,-7.071068e-01,-0.866004,-0.500037,13.585110
4,2019-09-01 12:00:00,17,14,3,11,5,0,7,4,33.22,1.334979e-11,-1.000000e+00,-0.867077,-0.498174,0.455399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3187,2020-11-30 06:00:00,7,7,8,2,0,11,1,2,0.01,1.000000e+00,6.094791e-13,-0.510105,0.860112,0.224339
3188,2020-11-30 09:00:00,9,9,8,0,1,10,13,2,1.84,7.071068e-01,-7.071068e-01,-0.508255,0.861207,0.224316
3189,2020-11-30 15:00:00,11,9,7,11,1,59,13,2,0.02,-7.071068e-01,-7.071068e-01,-0.504546,0.863385,0.224270
3190,2020-11-30 18:00:00,11,9,7,7,0,52,14,4,0.00,-1.000000e+00,-5.736167e-12,-0.502689,0.864468,0.224238


In [115]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    db="solar_forecasting_app"
)

mycursor = mydb.cursor()

In [116]:
#mycursor.execute("SELECT forecast_date from briton_ferry_forecasts WHERE forecast_date")
forecasted_date = time.strftime("2019-09-01 00:00:00")

mycursor.execute("UPDATE briton_ferry_forecasts SET actual_mv = 1, forecasted_mv = 10 WHERE forecast_date = %s", (forecasted_date,))
#mycursor.execute("SELECT * FROM briton_ferry_forecasts WHERE id=1")

myresult = mycursor.fetchall()


for row in myresult:
    print(row)
    print("\n")

In [117]:
mycursor.execute("SELECT * FROM briton_ferry_forecasts")

myresult = mycursor.fetchall()


for row in myresult:
    print(row)
    print("\n")


(1, datetime.datetime(2019, 9, 1, 0, 0), 10.0, 1.0, datetime.datetime(2022, 4, 22, 11, 58), datetime.datetime(2022, 4, 22, 15, 4, 55))


(2, datetime.datetime(2019, 9, 1, 3, 0), 34.553062438964844, 0.04, datetime.datetime(2022, 4, 22, 12, 43), datetime.datetime(2022, 4, 22, 15, 4, 55))




In [118]:
tempdf = df[0:0]

In [119]:
def checkIfDateExists(mycursor, forecast_date):
    forecast_date = time.strftime(forecast_date)
    query = ("""SELECT forecast_date FROM briton_ferry_forecasts WHERE  forecast_date = %s""")
    mycursor.execute(query, (forecast_date,))

    return mycursor.fetchone() is not None

In [120]:
def updateRow(mycursor, forecast_date, forecasted_mv, actual_mv):
    print(forecasted_mv, actual_mv)
    forecasted_date = time.strftime(forecast_date)
    now = datetime.datetime.utcnow()
    updated_at = now.strftime("%Y-%m-%d %H:%m:%S")
    query = ("""UPDATE briton_ferry_forecasts
                SET forecast_date = %s,
                    forecasted_mv = %s,
                    actual_mv = %s,
                    updated_at = %s
                WHERE forecast_date = %s""")
    varsToUpdate = (forecasted_date, forecasted_mv, actual_mv, updated_at, forecasted_date)
    mycursor.execute(query, varsToUpdate)

In [121]:
def insertForecasts(mycursor, forecast_date, forecasted_mv, actual_mv):
    now = datetime.datetime.utcnow()
    updated_at = now.strftime("%Y-%m-%d %H:%m:%S")
    created_at = now.strftime("%Y-%m-%d %H:%m:%S")
    insertQuery = ("""INSERT INTO briton_ferry_forecasts (forecast_date, forecasted_mv, actual_mv, updated_at, created_at) 
                        VALUES(%s,%s,%s,%s,%s)""")

    rowsToInsert = (forecast_date, forecasted_mv, actual_mv, updated_at, created_at)

    mycursor.execute(insertQuery, rowsToInsert)

In [122]:
def updatedb(df, mycursor, tempdf):  
    for i, row in df.iterrows():
        if checkIfDateExists(mycursor, row['forecastPeriod']):
            updateRow(mycursor, row['forecastPeriod'], row['Predictions'], row['AO_PV_generation'])
        else:
            tempdf = tempdf.append(row)
    return tempdf

In [123]:
newForecastsDf = updatedb(df, mycursor, tempdf)

for i, row in newForecastsDf.iterrows():
    insertForecasts(mycursor, row['forecastPeriod'], row['Predictions'], row['AO_PV_generation'])

10.819783210754396 0.0
34.553062438964844 0.04


/var/folders/sk/rz5f8_3s0v32jjstsjn8v30m0000gn/T/ipykernel_914/323552970.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempdf = tempdf.append(row)
/var/folders/sk/rz5f8_3s0v32jjstsjn8v30m0000gn/T/ipykernel_914/323552970.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempdf = tempdf.append(row)
/var/folders/sk/rz5f8_3s0v32jjstsjn8v30m0000gn/T/ipykernel_914/323552970.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempdf = tempdf.append(row)
/var/folders/sk/rz5f8_3s0v32jjstsjn8v30m0000gn/T/ipykernel_914/323552970.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tempdf = tempdf.append(row)
/var/folders/sk/rz5f8_3s0v32jjst

In [124]:
mydb.commit()
mydb.close()